# Why are old visits changing.
Every time ops updates the Ops plan we are noticing that jobs that have flight dates months in the past are changing. This notebook is an attempt to find out why

In [1]:
#import standard libraries (ie pandas)
from base import *

Using DJANGO_SETTINGS_MODULE = app.settings.dev
base_dir:  /home/wroscoe/code/works


### Import data 

In [2]:
#Import visit data from august and today
df_old = pd.read_csv('~/Dropbox/ceres_ops/8-15_ops_data.csv', parse_dates=['flight__date'])
df = pd.read_csv('~/Dropbox/ceres_ops/10-11_ops_data.csv', parse_dates=['flight__date'])

In [3]:
#Join the two datasets on their visit ids and only select the columns we want to look at.
dfs = []
dfc = pd.merge(df[['id', 'flight__date', 'status__name']], 
               df_old[['id', 'flight__date', 'status__name']], 
               how='outer', on='id')


In [4]:
#rename the columns so they make sense.
dfc = dfc.rename(columns={'id': 'visit_id', 
                        'flight__date_x': 'flight__date_new',
                        'status__name_x': 'status__name_new', 
                        'flight__date_y': 'flight__date_old', 
                        'status__name_y': 'status__name_old'})
dfc.head()

,visit_id,flight__date_new,status__name_new,flight__date_old,status__name_old
0,10502,2017-06-14,Published (partial),2017-06-14,Published (partial)
1,14,2014-04-23,None,2014-04-23,None
2,27,2014-05-01,None,2014-05-01,None
3,28,2014-05-07,None,2014-05-07,None
4,29,2014-05-07,None,2014-05-07,None


### Check status differences
We want to see what statuses of flights that have old flight dates have changed.

In [5]:
#select only the visits that have a flight date before july 1st
dfcf = dfc[dfc['flight__date_old'] < pd.to_datetime('2017-07-01')]
print('visits before filter {}'.format(len(dfc)))
print('visits after filter {}'.format(len(dfcf)))

visits before filter 15277
visits after filter 9256


In [6]:
#get only the visits where the new status is different than the old status
df_diff_statues = dfcf[dfcf['status__name_new'] != dfcf['status__name_old']]
df_diff_statues.head()

,visit_id,flight__date_new,status__name_new,flight__date_old,status__name_old
126,11863,2017-06-13,Published,2017-06-13,Flight Scheduled
127,11869,2017-06-13,Published,2017-06-13,Flight Scheduled
128,11775,2017-06-09,Published,2017-06-09,Flight Scheduled
298,14233,2017-09-14,Published,1900-01-01,Contract Submitted
458,8873,1900-01-01,Canceled,1900-01-01,Contract Submitted


In [7]:
#group the visits by their old and new status to see where most of the status differences are
df_diff_statues.groupby(['status__name_old', 'status__name_new']).count()

visit_id  flight__date_new  \
status__name_old   status__name_new                                       
Contract Submitted Canceled                        84                84   
                   Failed - Did Not Cover          40                40   
                   Failed - Flight Planning         6                 6   
                   Failed - Hardware                6                 6   
                   Failed - Other                   2                 2   
                   Failed - Pilot                   2                 2   
                   Failed - Weather                51                51   
                   Flight Scheduled                41                41   
                   Missed                           7                 7   
                   Preprocessing                    6                 6   
                   Published                      290               290   
                   Published (partial)             36                36   
Failed - Hardware  Failed - Weather                 1                 1   
Flight Planning    Canceled                         1                 1   
Flight Scheduled   Canceled                        27                27   
                   Failed - Other                   1                 1   
                   Published                       56                56   
Missed             Canceled                        62                62   
None               Canceled                         1                 1   
                   Flight Scheduled                 1                 1   
                   Published                        1                 1   
Pending Sales      Failed - Did Not Cover           1                 1   
                   Failed - Weather                 1                 1   
                   Published                        9                 9   
Published          Canceled                         5                 5   
                   Flight Scheduled                 7                 7   
Rescheduled        Canceled                        17                17   
                   Published                        2                 2   
                   Published (partial)              1                 1   
Sale Imported      Canceled                        11                11   
                   Failed - Did Not Cover          22                22   
                   Failed - Flight Planning         3                 3   
                   Failed - Hardware               19                19   
                   Failed - Pilot                   3                 3   
                   Failed - Weather                36                36   
                   Flight Scheduled                 3                 3   
                   Preprocessing                    1                 1   
                   Published                      196               196   
                   Published (partial)             79                79   

                                             flight__date_old  
status__name_old   status__name_new                            
Contract Submitted Canceled                                84  
                   Failed - Did Not Cover                  40  
                   Failed - Flight Planning                 6  
                   Failed - Hardware                        6  
                   Failed - Other                           2  
                   Failed - Pilot                           2  
                   Failed - Weather                        51  
                   Flight Scheduled                        41  
                   Missed                                   7  
                   Preprocessing                            6  
                   Published                              290  
                   Published (partial)                     36  
Failed - Hardware  Failed - Weather                         1  
Flight Pl

### Check Flight Date differences

In [8]:
#get only the visits where the new status is different than the old status
df_diff_flightdates = dfcf[dfcf['flight__date_new'] != dfcf['flight__date_old']]
df_diff_flightdates.head()

,visit_id,flight__date_new,status__name_new,flight__date_old,status__name_old
298,14233,2017-09-14,Published,1900-01-01,Contract Submitted
721,8875,2017-10-13,Canceled,1900-01-01,Contract Submitted
923,8876,2017-11-15,Flight Scheduled,1900-01-01,Contract Submitted
1153,8879,2018-02-14,Flight Scheduled,1900-01-01,Contract Submitted
1677,1657,1900-01-01,Canceled,2015-09-25,Published


In [9]:
#group the visits by their old and new status to see where most of the status differences are
df_diff_flightdates.groupby(['flight__date_old', 'flight__date_new']).count()

visit_id  status__name_new  \
flight__date_old flight__date_new                               
1900-01-01       2017-08-02               1                 1   
                 2017-08-04               6                 6   
                 2017-08-05               8                 8   
                 2017-08-07              41                41   
                 2017-08-08              51                51   
                 2017-08-09              13                13   
                 2017-08-10              10                10   
                 2017-08-11              17                17   
                 2017-08-12               3                 3   
                 2017-08-14              52                52   
                 2017-08-15              25                25   
                 2017-08-16              50                50   
                 2017-08-17              32                32   
                 2017-08-18              24                24   
                 2017-08-19              20                20   
                 2017-08-20              12                12   
                 2017-08-22              45                45   
                 2017-08-23              22                22   
                 2017-08-24              11                11   
                 2017-08-25              27                27   
                 2017-08-26               4                 4   
                 2017-08-28              20                20   
                 2017-08-29              25                25   
                 2017-08-30              36                36   
                 2017-09-01              10                10   
                 2017-09-04               6                 6   
                 2017-09-05              41                41   
                 2017-09-06              25                25   
                 2017-09-07              29                29   
                 2017-09-09               6                 6   
                 2017-09-11              19                19   
                 2017-09-12              17                17   
                 2017-09-13              36                36   
                 2017-09-14              33                33   
                 2017-09-18              17                17   
                 2017-09-20               2                 2   
                 2017-09-25              13                13   
                 2017-09-26               8                 8   
                 2017-09-27               6                 6   
                 2017-09-30               1                 1   
                 2017-10-02              10                10   
                 2017-10-03               4                 4   
                 2017-10-09               1                 1   
                 2017-10-10              11                11   
                 2017-10-11               1                 1   
                 2017-10-13              17                17   
                 2017-10-17               1                 1   
                 2017-11-15               8                 8   
                 2017-12-15               8                 8   
                 2018-01-15               8                 8   
                 2018-02-14               1                 1   
2015-09-25       1900-01-01               3                 3   
2015-09-27       1900-01-01               1                 1   
2015-09-28       1900-01-01               2                 2   
2015-09-29       2015-09-22               3                 3   
2017-05-10       1900-01-01               1                 1   

                                   status__name_old  
flight__date_old flight__date_new                    
1900-01-01       2017-08-02                       1  
                 2017-08-04                       6  
                 2017-08-05                     

In [10]:
dfcf.to_csv('~/Dropbox/ceres_ops/visit_compare_diff_dates.csv')

### Spot check some of the differences